In [ ]:
from IPython.display import HTML
HTML(filename='/Users/hubatsl/Desktop/SPT/Us/SPT/Python/hide_code.html')

# Off rate databank for pkc-ts experiment

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys
import tifffile
import time
# Add path to python modules
sys.path.append(os.getcwd()[0:-7]+'src/') 
from MovieTracks import (DiffusionFitter, OffRateFitter, ParticleFinder)
# Plotting becomes part of the notebook
%matplotlib inline

**Run thresholding on movie movieNo, read and write threshold from excel file**

In [ ]:
xls_file = '/Users/hubatsl/Desktop/OffRateFigureFlorent/OffRateParams.xlsx'
xls_db = pd.read_excel(xls_file)
movieNo = 7

if np.isnan(xls_db['Threshold'][movieNo]):
    xls_db.loc[movieNo, 'Threshold']=1000
thresh = int(xls_db['Threshold'][movieNo])
if np.isnan(xls_db['StartFrame'][movieNo]):
    xls_db.loc[movieNo, 'StartFrame'] = 0
startFrame = int(xls_db.loc[movieNo, 'StartFrame'])
if np.isnan(xls_db['MaxSize'][movieNo]):
    xls_db.loc[movieNo, 'MaxSize'] = 0
maxSize = int(xls_db.loc[movieNo, 'MaxSize'])

def adj_thresh(calibFrame, thresh, maxSize, startFrame):
    calibFrame = int(calibFrame)
    thresh = int(thresh)
    startFrame = int(startFrame)
    maxSize = int(maxSize)
    if maxSize is not 0: 
        maxSize = int(maxSize)
        o = OffRateFitter(xls_db['Folder'][movieNo], thresh, pixelSize=0.120,
                          maxsize=maxSize/10000, startFrame=startFrame)
    else:
        o = OffRateFitter(xls_db['Folder'][movieNo], thresh, pixelSize=0.120,
                          maxsize=None, startFrame=startFrame)
    print(xls_db['Folder'][movieNo])
    o.plot_calibration(calibrationFrame=calibFrame)
    xls_db.loc[movieNo, 'StartFrame'] = startFrame
    xls_db.loc[movieNo, 'Threshold'] = thresh
    xls_db.loc[movieNo, 'MaxSize'] = maxSize
    xls_db.to_excel(xls_file)

interact(adj_thresh, thresh=str(thresh), calibFrame='0', maxSize=str(maxSize),
         startFrame=str(startFrame), __manual=True)

**Read time stamps for each movie, write to excel file**

In [ ]:
xls_db['IntervalReal'] = 'nan'
xls_db['ExposureReal'] = 'nan'
xls_db['Length'] = 'nan'
for i, path in enumerate(xls_db['Folder']):
    p = ParticleFinder(path)
    xls_db.loc[i, 'IntervalReal'] = p.timestep
    xls_db.loc[i, 'ExposureReal'] = p.exposure
    xls_db.loc[i, 'Length'] = len(p.frames)
xls_db.to_excel(xls_file)

In [ ]:
offRates = []
bleachingRates = []
for i in range(len(xls_db['Folder'])):
    fol = xls_db.loc[i, 'Folder']
    threshold = xls_db.loc[i, 'Threshold']
    maxsize = xls_db.loc[i, 'MaxSize']
    if int(maxsize) is 0: maxsize=None
    o = OffRateFitter(filePath=fol, threshold=threshold, maxsize=maxsize,
                      parallel=False, pixelSize=0.12, timestep=2,
                      saveFigs=True, showFigs=True, autoMetaDataExtract=True)
    o.analyze()
    o.plot_calibration(calibrationFrame=0)
    o.fit_offRate([5])
    offRates.append(o.kOffVar5)
    bleachingRates.append(o.kPhVar5)
    

In [ ]:
offRates

In [ ]:
bleachingRates

In [ ]:
plt.plot(offRates, bleachingRates)
plt.show()